<a href="https://colab.research.google.com/github/camelliatea/proyek-analisis-sentimen/blob/main/Notebook_Pelatihan_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [45]:
import pandas as pd
import numpy as np

In [46]:
from sklearn.model_selection import train_test_split # Membagi data uji dan data latih
from sklearn.feature_extraction.text import TfidfVectorizer # untuk TF-IDF
from sklearn.feature_extraction.text import CountVectorizer # untuk BoW
from sklearn.linear_model import LogisticRegression # untuk LR
from sklearn.ensemble import RandomForestClassifier # untuk RF
from sklearn.metrics import accuracy_score # untuk menampilkan skor akurasi

In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Gathering Data

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
df = pd.read_csv('/content/drive/My Drive/data/ulasan_canva.csv')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_caseFoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity,text_length
0,976e8f99-a7bc-419c-9958-d260fe33b903,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Setelah update, Canva ada bug. Biasanya saat m...",3,1,2.306.0,2025-04-04 05:28:47,"Hai Natalia, terima kasih telah menghubungi Ca...",2025-04-06 02:32:24,2.306.0,Setelah update Canva ada bug Biasanya saat mau...,setelah update canva ada bug biasanya saat mau...,setelah update canva ada kesalahan dalam aplik...,"['setelah', 'update', 'canva', 'ada', 'kesalah...","['update', 'canva', 'kesalahan', 'aplikasi', '...",update canva kesalahan aplikasi pilih gambar o...,-13,negative,32
1,5274cee6-bcf0-4a64-832f-2453a91da93d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Menggunakan Canva pro memang banyak pilihannya...,4,32,2.305.0,2025-03-23 14:51:48,"Hai Sahroni, terima kasih telah menghubungi Ca...",2025-03-25 02:10:43,2.305.0,Menggunakan Canva pro memang banyak pilihannya...,menggunakan canva pro memang banyak pilihannya...,menggunakan canva pro memang banyak pilihannya...,"['menggunakan', 'canva', 'pro', 'memang', 'ban...","['canva', 'pro', 'pilihannya', 'template', 'de...",canva pro pilihannya template design pro bagus...,19,positive,17
2,82ec6264-f315-4b1f-aab8-b2e6042e100d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kecewa banget!! Padahal termasuk penggunaan se...,1,46,2.304.0,2025-03-15 09:35:01,"Hai Himatul, terima kasih kerana menghubungi C...",2025-03-17 06:25:18,2.304.0,Kecewa banget Padahal termasuk penggunaan seti...,kecewa banget padahal termasuk penggunaan seti...,kecewa banget padahal termasuk penggunaan seti...,"['kecewa', 'banget', 'padahal', 'termasuk', 'p...","['kecewa', 'banget', 'penggunaan', 'setia', 'c...",kecewa banget penggunaan setia canva masak bik...,12,positive,29
3,f9beaf06-1d97-43af-830a-c9606eb6c3a1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,TOLONG UNTUK DEVELOPER DIBACA!! sbenernya apli...,1,49,2.304.0,2025-03-15 14:34:10,"Halo Dnzdnsanss, terima kasih atas masukan And...",2025-03-17 06:06:29,2.304.0,TOLONG UNTUK DEVELOPER DIBACA sbenernya aplika...,tolong untuk developer dibaca sbenernya aplika...,tolong untuk developer dibaca sebenarnya aplik...,"['tolong', 'untuk', 'developer', 'dibaca', 'se...","['tolong', 'developer', 'dibaca', 'aplikasi', ...",tolong developer dibaca aplikasi bagus masuk a...,5,positive,19
4,a669246c-2df2-476f-a5e0-bc01dd6a8afb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Tolong tambahkan fitur sembunyikan seperti di ...,4,44,2.298.0,2025-02-22 23:48:46,"Hai Feldian, terima kasih telah menghubungi Ca...",2025-02-25 04:13:19,2.298.0,Tolong tambahkan fitur sembunyikan seperti di ...,tolong tambahkan fitur sembunyikan seperti di ...,tolong tambahkan fitur sembunyikan seperti di ...,"['tolong', 'tambahkan', 'fitur', 'sembunyikan'...","['tolong', 'tambahkan', 'fitur', 'sembunyikan'...",tolong tambahkan fitur sembunyikan pixellab mu...,11,positive,40


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108000 entries, 0 to 107999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              108000 non-null  object
 1   userName              108000 non-null  object
 2   userImage             108000 non-null  object
 3   content               108000 non-null  object
 4   score                 108000 non-null  int64 
 5   thumbsUpCount         108000 non-null  int64 
 6   reviewCreatedVersion  108000 non-null  object
 7   at                    108000 non-null  object
 8   replyContent          108000 non-null  object
 9   repliedAt             108000 non-null  object
 10  appVersion            108000 non-null  object
 11  text_clean            108000 non-null  object
 12  text_caseFoldingText  108000 non-null  object
 13  text_slangwords       107892 non-null  object
 14  text_tokenizingText   108000 non-null  object
 15  text_stopword    

In [51]:
# Menghapus missing values
df = df.dropna()

# Menghapus duplikasi
df = df.drop_duplicates()

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107321 entries, 0 to 107999
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              107321 non-null  object
 1   userName              107321 non-null  object
 2   userImage             107321 non-null  object
 3   content               107321 non-null  object
 4   score                 107321 non-null  int64 
 5   thumbsUpCount         107321 non-null  int64 
 6   reviewCreatedVersion  107321 non-null  object
 7   at                    107321 non-null  object
 8   replyContent          107321 non-null  object
 9   repliedAt             107321 non-null  object
 10  appVersion            107321 non-null  object
 11  text_clean            107321 non-null  object
 12  text_caseFoldingText  107321 non-null  object
 13  text_slangwords       107321 non-null  object
 14  text_tokenizingText   107321 non-null  object
 15  text_stopword         

# Data Splitting dan Ekstraksi Fitur

In [53]:
# Memisahkan kolom 'text_akhir' menjadi fitur (X) dan kolom 'polarity' menjadi target (y)
X = df['text_akhir']
y = df['polarity']

### **Ekstraksi Fitur**

**TF-IDF**

In [54]:
# Inisialisasi TfidfVectorizer dengan 1000 unigram dan bigram terpenting yang muncul minimal di 5 dokumen namun tidak lebih dari 85% dokumen
tfidf_vectorizer = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.85, ngram_range=(1,2))
# Transformasi menjadi vektor TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(X)
# Konversi menjadi Dataframe
X_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Melihat hasil
X_tfidf

,admin,akhir,akses,akun,akun belajar,akun google,alhamdulillah,alhamdulillah membantu,allah,ama,...,wajib unduh,warna,wattpad,web,whatsapp,wifi,wkwk,worth,wow,youtube
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.197574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.499564,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107317,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Bag of Words (BoW)**

In [55]:
# Inisialisasi vectorizer dengan 1000 fitur terpenting yang muncul minimal di 5 dokumen namun tidak lebih dari 85% dokumen
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.85)
# Konversi menjadi vektor BoW
bow_matrix = vectorizer.fit_transform(X)
# Konversi menjadi dataframe
X_bow = pd.DataFrame(bow_matrix.toarray(), columns=vectorizer.get_feature_names_out())

X_bow

,admin,ai,akhir,akses,akun,alasan,alat,alhamdulillah,allah,ama,...,wattpad,web,website,whatsapp,wifi,wkwk,worth,wow,yng,youtube
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107316,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
107317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107318,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107319,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Modeling

In [56]:
# Membagi data menjadi data latih dan data uji

# 80/20 TF-IDF
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# 70/30 TF-IDF
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# 80/20 BoW
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_bow, y, test_size=0.2, random_state=42)

In [57]:
# Logistic Regression | TF-IDF | 80/20
lr = LogisticRegression()
lr.fit(X_train1, y_train1)

y_train_pred1 = lr.predict(X_train1)
y_test_pred1 = lr.predict(X_test1)

result1 = {
    'Model': 'Logistic Regression (TF-IDF 80/20)',
    'Train Accuracy': accuracy_score(y_train1, y_train_pred1),
    'Test Accuracy': accuracy_score(y_test1, y_test_pred1)
}

In [58]:
# Random Forest | TF-IDF | 70/30
rf_tfidf = RandomForestClassifier()
rf_tfidf.fit(X_train2, y_train2)

y_train_pred2 = rf_tfidf.predict(X_train2)
y_test_pred2 = rf_tfidf.predict(X_test2)

result2 = {
    'Model': 'Random Forest (TF-IDF 70/30)',
    'Train Accuracy': accuracy_score(y_train2, y_train_pred2),
    'Test Accuracy': accuracy_score(y_test2, y_test_pred2)
}

In [59]:
# Random Forest | BoW | 80/20
rf_bow = RandomForestClassifier()
rf_bow.fit(X_train3, y_train3)

y_train_pred3 = rf_bow.predict(X_train3)
y_test_pred3 = rf_bow.predict(X_test3)

result3 = {
    'Model': 'Random Forest (BoW 80/20)',
    'Train Accuracy': accuracy_score(y_train3, y_train_pred3),
    'Test Accuracy': accuracy_score(y_test3, y_test_pred3)
}

In [60]:
# Menampilkan hasil dalam bentuk dataframe
results = [result1, result2, result3]
accuracy_df = pd.DataFrame(results)
print(accuracy_df)

                                Model  Train Accuracy  Test Accuracy
0  Logistic Regression (TF-IDF 80/20)        0.947563       0.941672
1        Random Forest (TF-IDF 70/30)        0.984266       0.922291
2           Random Forest (BoW 80/20)        0.989517       0.925134


### **Algoritma Deep Learning LSTM**

In [61]:
# Mengubah kolom polarity yang berupa kategori menjadi numerik
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Menyimpan jumlah kelas unik dari y
num_classes = len(label_encoder.classes_)

# Tokenisasi, membatasi hanya 5000 kata teratas, sisanya jadi <OOV> (out-of-vocabulary)
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X)

# Mengubah setiap teks menjadi urutan angka
sequences = tokenizer.texts_to_sequences(X)

# Menyamakan panjang semua urutan menjadi 20 token, padding di belakang
X = pad_sequences(sequences, maxlen=20, padding='post')

# Mengubah y menjadi one-hot encoding untuk loss function categorical_crossentropy
y = to_categorical(y, num_classes=num_classes)

# Membagi data untuk training dan testing 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
# Membangun arsitektur LSTM
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64))  # Layer embedding: mengonversi token ke vektor 64 dimensi
model.add(LSTM(64, return_sequences=False))          # LSTM layer, hanya output akhir yang diambil
model.add(Dropout(0.5))                              # Dropout 50% untuk mencegah overfitting
model.add(Dense(3, activation='softmax'))            # Output layer dengan 3 neuron (sesuai jumlah kelas)

In [63]:
# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [64]:
# Melatih model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1)

Epoch 1/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 28s 21ms/step - accuracy: 0.8365 - loss: 0.4592 - val_accuracy: 0.9489 - val_loss: 0.1600
Epoch 2/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.9636 - loss: 0.1102 - val_accuracy: 0.9595 - val_loss: 0.1238
Epoch 3/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 42s 21ms/step - accuracy: 0.9741 - loss: 0.0819 - val_accuracy: 0.9646 - val_loss: 0.1122
Epoch 4/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.9791 - loss: 0.0670 - val_accuracy: 0.9635 - val_loss: 0.1144
Epoch 5/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.9804 - loss: 0.0597 - val_accuracy: 0.9666 - val_loss: 0.1099
Epoch 6/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 39s 19ms/step - accuracy: 0.9842 - loss: 0.0518 - val_accuracy: 0.9649 - val_loss: 0.1319
Epoch 7/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 43s 20ms/step - accuracy: 0.9854 - loss: 0.0477 - val_accuracy: 0.9626 - val_loss: 0.1395
Epoch 8/10
1208/1208 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9869 -

In [65]:
# Melakukan prediksi untuk training dan testing
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Mengambil kelas dengan probabilitas tertinggi
y_train_pred_labels = np.argmax(y_train_pred, axis=1)
y_test_pred_labels = np.argmax(y_test_pred, axis=1)
y_train_true_labels = np.argmax(y_train, axis=1)
y_test_true_labels = np.argmax(y_test, axis=1)

# Mengevaluasi akurasi
train_acc = accuracy_score(y_train_true_labels, y_train_pred_labels)
test_acc = accuracy_score(y_test_true_labels, y_test_pred_labels)

2683/2683 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step
671/671 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step


In [101]:
# Menyimpan hasil evaluasi ke dalam DataFrame
result4 = {
    'Model': 'Long Short-Term Memory (80/20)',
    'Train Accuracy': train_acc,
    'Test Accuracy': test_acc
}
accuracy_df = pd.DataFrame([result4])
print(accuracy_df)

                            Model  Train Accuracy  Test Accuracy
0  Long Short-Term Memory (80/20)        0.988632       0.962357


# **Inference**

In [102]:
final_results = [result1, result2, result3, result4]
accuracy_df = pd.DataFrame(final_results).sort_values(by='Test Accuracy', ascending=False)
print(accuracy_df)

                                Model  Train Accuracy  Test Accuracy
3      Long Short-Term Memory (80/20)        0.988632       0.962357
0  Logistic Regression (TF-IDF 80/20)        0.947563       0.941672
2           Random Forest (BoW 80/20)        0.989517       0.925134
1        Random Forest (TF-IDF 70/30)        0.984266       0.922291


Dalam Analisis Sentimen ini, algoritma LSTM mencapai akurasi tertinggi sebesar 96,23%. Oleh karena itu, algoritma LSTM dipilih sebagai model utama yang akan digunakan untuk memprediksi sentimen pada kalimat baru.

In [113]:
# Fungsi untuk memprediksi sentimen dari teks input menggunakan algoritma LSTM
def predict_lstm(text):
  seq = tokenizer.texts_to_sequences([text])
  padded = pad_sequences(seq, maxlen=20, padding='post')
  probs = model.predict(padded)
  pred_class = np.argmax(probs, axis=1)[0] # Mengambil nilai pertama dari array hasil argmax
  return label_encoder.inverse_transform([pred_class])[0]

# Contoh testing
input_text = 'Aplikasi sering keluar sendiri tanpa alasan.'

# Melakukan prediksi
lstm_model = predict_lstm(input_text)

print(f'Hasil Prediksi LSTM: {lstm_model}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Hasil Prediksi LSTM: negative
